# Создание и обучение нейросети в Keras

И вот мы дошли до точки перехода от теории к практике. Соберём и обучим нейросеть на датасете MNIST для распознавания рукописных цифр.

## MNIST

MNIST - классический датасет для задачи классификации. Состоит из набора чёрно-белых изображений (28х28) с рукописными цифрами. Всего получается 10 классов (логишно). Чтобы подгрузить MNIST достаточно выполнить код:

```Python
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
```

In [1]:
import matplotlib.pyplot as plt
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

fig, axes = plt.subplots(3, 5, figsize=(20, 12));
for i, ax in enumerate(axes.ravel()):
    ax.imshow(x_train[i], cmap='gray_r')

plt.show();

/home/dmitry/.pyenv/versions/3.6.4/lib/python3.6/site-packages/h5py-2.7.1-py3.6-linux-x86_64.egg/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


<Figure size 2000x1200 with 15 Axes>

## Создание модели (Sequential)

Первый способ создания модели - это Sequential. Такая модель представляет собой просто последовательное соединение слоёв, один за другим. Со вторым способом задания модели познакомимся в главе про свёрточные сети. 

Чтобы собрать Sequential модель нужно создать объект типа `keras.models.Sequential` и последовательно добавлять слои и активации:

In [23]:
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.datasets import mnist
from keras.optimizers import SGD, Adam
from keras.losses import categorical_crossentropy
from keras.metrics import categorical_accuracy

# Поскольку у сети 10 выходов, необходимо вектор из N ответа представить в виде [N, 10]. Такое кодирование называется One Hot Encoding (OHE). 
# Например, в случае трёх признаков вектор ответов [0, 2, 1] превратится в [[1, 0, 0], [0, 0, 1], [0, 1, 0]].
def OHE(x):
    y = np.zeros(shape=(x.size, 10))
    y[np.arange(x.size), x] = 1
    return y

# Подгружаем датасет
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# И начинаем собирать модель:
model = Sequential()

# Добавляем первый полносвязный слой. 
model.add(Dense(units=64,  # Количество нейронов слоя
                input_dim=28*28,  # Не забываем для первого слоя явно указать размер входного вектора. 
                kernel_initializer='he_normal', # И про инициализацию весов.
                use_bias=True)) # Можем добавить нейрон смещения
model.add(Activation('relu'))  # Так же не забываем про активационную функцию
model.add(Dropout(0.2))  # Добавим Dropout чтобы не оверфитнуться

model.add(Dense(10))  # И добавим выходной слой. Количество нейронов должно совпадать с количеством классов.
model.add(Activation('softmax'))  # Не забываем про softmax. Без него категориальная кроссэнтропия не работает (см. главу про функции потерь)

# Выбираем оптимизатор, функцию потерь и метрики
# optimizer = SGD(lr=0.01, momentum=0.8, nesterov=True)
optimizer = Adam(lr=0.005, amsgrad=True)
loss = categorical_crossentropy
metrics = [categorical_accuracy]  # Метрики задаются списком. Стало быть, можно использовать несколько.

# После чего собираем модель
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Осталось подготовить данные:
x_train = x_train.reshape(x_train.shape[0], -1)  # Сделаем из картинок плоские вектора
x_train = x_train.astype(float) / 255  # Отнормируем вектор признаков от 0 до 1. 
y_train_OHE = OHE(y_train)  # И закодируем вектор ответов при помощи OHE

# Сделаем то же самое с валидационными данными:
x_test = x_test.reshape(x_test.shape[0], -1)
x_test = x_test.astype(float) / 255
y_test_OHE = OHE(y_test)

# Всё, модель можно обучать:
model.fit(x_train, y_train_OHE, 
          epochs=10, 
          batch_size=64, 
          validation_data=(x_test, y_test_OHE));

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 3s 46us/step - loss: 0.3019 - categorical_accuracy: 0.9087 - val_loss: 0.1460 - val_categorical_accuracy: 0.9562
Epoch 2/10
60000/60000 [==============================] - 3s 43us/step - loss: 0.1844 - categorical_accuracy: 0.9435 - val_loss: 0.1334 - val_categorical_accuracy: 0.9580
Epoch 3/10
60000/60000 [==============================] - 3s 44us/step - loss: 0.1570 - categorical_accuracy: 0.9524 - val_loss: 0.1157 - val_categorical_accuracy: 0.9632
Epoch 4/10
60000/60000 [==============================] - 3s 42us/step - loss: 0.1383 - categorical_accuracy: 0.9563 - val_loss: 0.1168 - val_categorical_accuracy: 0.9673
Epoch 5/10
60000/60000 [==============================] - 3s 43us/step - loss: 0.1280 - categorical_accuracy: 0.9590 - val_loss: 0.1038 - val_categorical_accuracy: 0.9712
Epoch 6/10
60000/60000 [==============================] - 3s 42us/step - loss: 0.1199 - categor

А теперь пришло время проверить обученную модель и посмотрим на каких примерах модель промахнулась:

In [ ]:
import matplotlib.pyplot as plt
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Масштабируем тестовые признаки
x_test = x_test.reshape(x_test.shape[0], -1).astype('float')
x_test /= 255

# Прогоняем новые признаки через сеть методом model.predict()
pred = model.predict(x_test)

# Выходной вектор выглядит выглядит как [[p_10, p_11, ... p_19], [p_20, p_21, ...], ...[...p_N0]], где p_ij - вероятность пренадлежность объекта i классу j
predicted_class = np.argmax(pred, axis=1) # Приведём его к виду [class_1, class_2, ..., class_N]

match = predicted_class == y_test # Оценим точность как количество правильно угаданных классов
mismatch = ~match

print(f'Total mismatches: {mismatch.sum()}; Accuracy = {match.mean() * 100}%')
x = x_test[mismatch].reshape(-1, 28, 28)
y = y_test[mismatch]
p = predicted_class[mismatch]

fig, axes = plt.subplots(3, 6, figsize=(24, 12));
samples = np.random.choice(np.arange(y.size), axes.size, replace=False)
for i, ax in zip(samples, axes.ravel()):
    ax.imshow(x[i], cmap='gray_r')
    ax.set_title(f'Predicted as {p[i]}. True labels is {y[i]}')
    
plt.show();